# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [109]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/jinshuo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jinshuo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jinshuo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jinshuo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [94]:
# import libraries
import sys
import re
import pandas as pd

from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [124]:
engine.table_names()[0]

'DisasterResponse.db'

In [126]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table(engine.table_names()[0],con=engine)

X = df.message
y = df.iloc[:,4:]

cat_names = y.columns

#for col in y.columns:
 #   y.loc[y[col] > 1, col] = 1

### 2. Write a tokenization function to process your text data

In [127]:
def tokenize(text):
    
    # normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize sentence
    token = word_tokenize(text)
    # remove stopwords and lemmatize 
    words = [WordNetLemmatizer().lemmatize(w).strip() for w in token if w not in stopwords.words('english')]

    return words

In [117]:
def tokenize(text):
    
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [111]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
y_pred = model.predict(X_test)

In [128]:
def build_model():
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    parameters = {'vect__ngram_range':[(1,2),(2,2)],
                  'clf__estimator__n_estimators':[3, 5]}

    cv = GridSearchCV(estimator = pipeline, param_grid=parameters)

    return cv

In [129]:
model = build_model()

In [130]:
model.fit(X_train, y_train)

/Users/jinshuo/anaconda/envs/Python35/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


TypeError: unorderable types: int() < str()

In [37]:
y_train.apply(lambda col: col.value_counts())

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0.0,1229,4345,5211,3008,4794,4963,5092,5140,5044,5234.0,...,5173,4989,3782,4789,4733,5190,4741,5129,4969,4253
1.0,4005,889,23,2226,440,271,142,94,190,NaN,...,61,245,1452,445,501,44,493,105,265,981


In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
Y_pred = pipeline.predict(X_test)
print(classification_report(y_test, Y_pred, target_names = category_names))
print('---------------------------------')

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      4978
               request       0.79      0.45      0.57      1125
                 offer       0.00      0.00      0.00        27
           aid_related       0.74      0.60      0.67      2664
          medical_help       0.48      0.12      0.19       452
      medical_products       0.81      0.11      0.19       317
     search_and_rescue       0.47      0.05      0.09       183
              security       0.00      0.00      0.00       119
              military       0.51      0.09      0.15       210
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.38      0.52       406
                  food       0.87      0.53      0.66       759
               shelter       0.79      0.24      0.37       589
              clothing       0.80      0.08      0.15        97
                 money       0.67      

/Users/jinshuo/anaconda/envs/Python35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/jinshuo/anaconda/envs/Python35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

/Users/jinshuo/anaconda/envs/Python35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.

/Users/jinshuo/anaconda/envs/Python35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.



In [16]:
def performance(model, X_test, y_test):
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print('Label: {}'.format(col))
        print('------------------------------------------------------')
        print(classification_report(y_test[col], y_pred[:, i]))

performance(pipeline, X_test, y_test)

Label: related
------------------------------------------------------
              precision    recall  f1-score   support

           0       0.64      0.45      0.53      1576
           1       0.84      0.92      0.88      4978

   micro avg       0.81      0.81      0.81      6554
   macro avg       0.74      0.69      0.70      6554
weighted avg       0.79      0.81      0.79      6554

Label: request
------------------------------------------------------
              precision    recall  f1-score   support

           0       0.90      0.98      0.93      5429
           1       0.79      0.45      0.57      1125

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.84      0.71      0.75      6554
weighted avg       0.88      0.89      0.87      6554

Label: offer
------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      

/Users/jinshuo/anaconda/envs/Python35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6268
           1       0.25      0.00      0.01       286

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.60      0.50      0.49      6554
weighted avg       0.93      0.96      0.94      6554

Label: weather_related
------------------------------------------------------
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      4691
           1       0.83      0.63      0.72      1863

   micro avg       0.86      0.86      0.86      6554
   macro avg       0.85      0.79      0.81      6554
weighted avg       0.86      0.86      0.85      6554

Label: floods
------------------------------------------------------
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6018
           1       0.88      0.40      0.55       536

   micro avg       0.95      0.95  

In [18]:
from sklearn.metrics import accuracy_score
y_pred_base = pipeline.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred_base))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred_base, average='weighted')))

Testing accuracy 0.2523649679584986


NameError: name 'f1_score' is not defined

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Check the list of available parameters with `estimator.get_params().keys()`.
list(pipeline.get_params().keys())

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'vect__ngram_range':[(1,2),(2,2)],
              
              'clf__estimator__n_estimators':[10, 30, 50]}

gs_clf = GridSearchCV(estimator = pipeline, param_grid=parameters)

In [ ]:
gs_clf.fit(X_train,y_train)

In [ ]:
y_pred_gs = gs_clf.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred_gs))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred_gs, average='weighted')))

In [ ]:
best_parameters = gs_clf.best_params_
best_accuracy = gs_clf.best_score_
print(best_parameters)
print(best_accuracy)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = gs_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,f1_score

In [ ]:
performance(gs_clf, X_test, y_test)

- After tune parameters of the model using grid search. The precision, recall, and f1-score to predict each label as 1 are much better than the base model previously.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
def new_model_pipeline():
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    return pipeline

In [ ]:
new_model_pipeline = new_model_pipeline()

In [ ]:
new_model_pipeline.get_params().keys()

In [ ]:
new_model_pipeline.fit(X_train, y_train)

In [ ]:
performance(new_model_pipeline, X_test, y_test)

### 9. Export your model as a pickle file

In [ ]:
from sklearn.externals import joblib

In [ ]:
joblib.dump(gs_clf, 'classification.pkl')


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [6]:
import json
import plotly
import pandas as pd

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from flask import Flask
from flask import render_template, request, jsonify
from plotly.graph_objs import Bar
from sklearn.externals import joblib
from sqlalchemy import create_engine
